In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['reggaeton']
YEARS = [2010, 2024] 

In [9]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')


In [19]:
# Años para iterar
YEARS = [2010, 2024] 

# Términos optimizados para una búsqueda de ALTO VOLUMEN
# Incluye etiquetas históricas ('dembow') y modernas ('trap latino', 'urbano latino')
query_terms = [
    'reggaeton',
    'regueton',
    'urbano latino',
    'perreo',
    'reggaeton pop',
    'latin urban',
    'pop latino',
    'trap latino',
    'urbano latino',
    'dembow',
    '"hip hop latino"' # Entre comillas para buscar la frase exacta (útil para el 2000)
]

# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

✅ Conexión a Spotify exitosa.


In [20]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [23]:
# --- DEFINICIÓN DE LA FUNCIÓN CON DOBLE BUCLE (MÁXIMA ROBUSTEZ Y LÍMITES) ---

def extract_spotify_data_reggaeton_ampliada(sp, years, query_terms): 
    """
    VERSION FINAL LIMITADA: Itera primero por AÑO y luego por CADA TÉRMINO de género.
    Máximo total: 500 canciones únicas. Máximo por año: 250 canciones únicas.
    """
    
    spotify_data = []
    MAX_TOTAL_SONGS = 500 # Nuevo límite global
    MAX_SONGS_PER_YEAR = 250 # Nuevo límite por año
    
    for year in years:
        
        # 1. VERIFICACIÓN DEL LÍMITE GLOBAL ANTES DE EMPEZAR EL AÑO
        if len(spotify_data) >= MAX_TOTAL_SONGS:
            print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Saltando el año {year}.")
            break
            
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")
        
        songs_added_in_year = 0 # Contador para este año
        
        # BUENO 1: Iteramos sobre cada término de búsqueda 
        for term in query_terms:
            
            # 2. VERIFICACIÓN DEL LÍMITE ANUAL
            if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                print(f"🛑 LÍMITE ANUAL ALCANZADO ({MAX_SONGS_PER_YEAR}). Terminando la búsqueda para el año {year}.")
                break
            
            query = f'{term} year:{year_range}'
            print(f"   -> Buscando con término: {term}")
            
            try:
                # La API siempre trae un máximo de 50 por llamada
                results = sp.search(q=query, type='track', limit=50) 
                
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")
                
                songs_added_in_term = 0
                
                # BUENO 2: Bucle de Paginación
                while results: # Mientras haya más resultados (no ponemos un límite aquí para exprimir los 1000)
                    
                    for track in results['tracks']['items']:
                        
                        track_id = track['id']
                        
                        # 3. VERIFICACIÓN DE LÍMITES DENTRO DEL BUCLE INTERNO
                        
                        # A. Límite Global alcanzado
                        if len(spotify_data) >= MAX_TOTAL_SONGS:
                            print(f"🛑 LÍMITE GLOBAL ({MAX_TOTAL_SONGS}) alcanzado. Terminando.")
                            return pd.DataFrame(spotify_data) # Terminamos inmediatamente
                            
                        # B. Límite Anual alcanzado
                        if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                            break # Salta al siguiente término de búsqueda
                        
                        # C. Deduplicación
                        # Usamos una forma más eficiente de comprobar duplicados si el volumen es grande
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue
                            
                        # 4. Añadir la canción
                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'Reggaeton (Origen: {term})',
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        songs_added_in_term += 1
                        songs_added_in_year += 1 # Contar para el límite anual

                    # Paginación
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                    else:
                        results = None
                    
                    # Aumentamos la pausa para evitar el Rate Limiting
                    time.sleep(1.5) 
                
                print(f"      -> Añadidas {songs_added_in_term} canciones ÚNICAS de '{term}'.")
                
            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(3) # Pausa más larga después de un error para recuperarse
            
    return pd.DataFrame(spotify_data)

In [24]:
# --- LLAMADA AL CÓDIGO (CELDA 3 CORREGIDA) ---

if sp:
    # Eliminamos el argumento 'max_total_songs=500'
    df_spotify_raw = extract_spotify_data_reggaeton_ampliada(sp, YEARS, query_terms) 
    
    print("\n--- RESULTADO FINAL DE REGGAETON (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}") 
    print(df_spotify_raw.head(10))


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: reggaeton
   -> Spotify reporta 100 resultados para 'reggaeton'.
      -> Añadidas 100 canciones ÚNICAS de 'reggaeton'.
   -> Buscando con término: regueton
   -> Spotify reporta 32 resultados para 'regueton'.
      -> Añadidas 19 canciones ÚNICAS de 'regueton'.
   -> Buscando con término: urbano latino
   -> Spotify reporta 36 resultados para 'urbano latino'.
      -> Añadidas 34 canciones ÚNICAS de 'urbano latino'.
   -> Buscando con término: perreo
   -> Spotify reporta 81 resultados para 'perreo'.
      -> Añadidas 68 canciones ÚNICAS de 'perreo'.
   -> Buscando con término: reggaeton pop
   -> Spotify reporta 29 resultados para 'reggaeton pop'.
      -> Añadidas 23 canciones ÚNICAS de 'reggaeton pop'.
   -> Buscando con término: latin urban
   -> Spotify reporta 100 resultados para 'latin urban'.
      -> Añadidas 6 canciones ÚNICAS de 'latin urban'.
🛑 LÍMITE ANUAL ALCANZADO (250). Terminando la búsqueda pa

In [25]:
# VERIFICACIÓN DE CANCIONES
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 500
Primeras filas (head) de df_spotify_raw:
          Artista                 Género musical     Tipo  \
0          Plan B  Reggaeton (Origen: reggaeton)  Canción   
1  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
2     Cosculluela  Reggaeton (Origen: reggaeton)  Canción   
3         Farruko  Reggaeton (Origen: reggaeton)  Canción   
4        Arcángel  Reggaeton (Origen: reggaeton)  Canción   

                          Nombre  Año de lanzamiento              ID_Spotify  \
0  Parisera (feat. De La Ghetto)                2010  4rVeA8n8ui7neX4XUOJObT   
1                        Tu Olor                2010  4lwJGmQFcbUypQ8mkm8YlG   
2                Te Deseo el Mal                2010  43hrb3UDtUdvVNQ5gSCsMI   
3               Su Hija Me Gusta                2010  5N1EdvOZKDKKIcwBcj3VmR   
4              Mujer Maravillosa                2010  6mybRTRWrEecDQWVSdjRrU   

                 ID_Album  
0  4s9iYsrtBIvkZn5AEbUbl0  


In [26]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

# Asegúrate de que las librerías necesarias (pandas, time) y la variable 'sp' estén disponibles
import pandas as pd
import time
# Nota: La variable 'sp' debe estar definida por la Celda 1.

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 291 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 291 ÁLBUMES ---
✅ Extracción de 291 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 500 filas.
          Artista                 Género musical     Tipo  \
0          Plan B  Reggaeton (Origen: reggaeton)  Canción   
1  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
2     Cosculluela  Reggaeton (Origen: reggaeton)  Canción   
3         Farruko  Reggaeton (Origen: reggaeton)  Canción   
4        Arcángel  Reggaeton (Origen: reggaeton)  Canción   

                          Nombre  Año de lanzamiento              ID_Spotify  \
0  Parisera (feat. De La Ghetto)                2010  4rVeA8n8ui7neX4XUOJObT   
1                        Tu Olor                2010  4lwJGmQFcbUypQ8mkm8YlG   
2                Te Deseo el Mal                2010  43hrb3UDtUdvVNQ5gSCsMI 

In [27]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None

# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_reggaeton = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )
    
    print("\n--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---")
    print(df_final_reggaeton.head())
    print(f"Total de registros finales para Reggaeton: {len(df_final_reggaeton)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE REGGAETON LISTO PARA ALMACENAMIENTO ---
          Artista                 Género musical     Tipo  \
0          Plan B  Reggaeton (Origen: reggaeton)  Canción   
1  Wisin & Yandel  Reggaeton (Origen: reggaeton)  Canción   
2     Cosculluela  Reggaeton (Origen: reggaeton)  Canción   
3         Farruko  Reggaeton (Origen: reggaeton)  Canción   
4        Arcángel  Reggaeton (Origen: reggaeton)  Canción   

                          Nombre  Año de lanzamiento              ID_Spotify  \
0  Parisera (feat. De La Ghetto)                2010  4rVeA8n8ui7neX4XUOJObT   
1                        Tu Olor                2010  4lwJGmQFcbUypQ8mkm8YlG   
2                Te Deseo el Mal                2010  43hrb3UDtUdvVNQ5gSCsMI   
3               Su Hija Me Gusta                2010  5N1EdvOZKDKKIcwBcj3VmR   
4              Mujer Maravillosa                2010  6mybRTRWrEecDQWVSdjRrU   

                 ID_Album Playcount_LastFM Listener

In [ ]:
# --- BLOQUE DE FUSIÓN Y GUARDADO CORREGIDO ---

# 1. Aseguramos que tenemos df_spotify_final y df_lastfm en memoria.

if 'df_spotify_final' in locals() and 'df_lastfm' in locals():
    
    # Lista de columnas a fusionar (¡EXCLUYE Artistas_Similares!)
    lastfm_cols_to_merge = ['Artista', 'Biografia_Resumen', 'Playcount_LastFM', 'Listeners_LastFM']
    
    # 2. Re-ejecutar el merge (fusión)
    df_proyecto_final = pd.merge(
        df_spotify_final, 
        # Seleccionamos solo las columnas que sabemos que existen:
        df_lastfm[lastfm_cols_to_merge], # <-- ¡CRUCIAL!
        on='Artista', 
        how='left'
    )
    
    # 3. Guardar el archivo CSV (Tu Celda 5 original)
    output_filename = 'reggaeton_data_2000_2024.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False,        
        encoding='utf-8'    
    )
    
    print(f"\n🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉")
# ...


🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉


In [16]:
!pip install mysql-connector-python
# O usa el que suele ser más robusto con Pandas:
!pip install sqlalchemy mysqlclient

FASE 2  

In [10]:
# --- CELDA 1 CORREGIDA: CREACIÓN Y CONEXIÓN A MYSQL ---
# --- CELDA 1: CONFIGURACIÓN Y CONEXIÓN A MYSQL (VERIFICACIÓN DE IMPORTACIONES) ---

import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, BigInteger, Text, VARCHAR # <--- ¡ESTAS!
from sqlalchemy.sql import text # <--- ¡ESTA!
import mysql.connector 
import numpy as np # <--- ¡ESTA!
# ... (el resto de tu código de conexión sigue aquí)

# 1. Cargar las credenciales y definir el motor de conexión
load_dotenv('2.env') 

CSV_FILENAME = 'reggaeton_data_2000_2024.csv'

DB_USER = os.getenv("MYSQL_USER")
DB_PASSWORD = os.getenv("MYSQL_PASSWORD")
DB_HOST = os.getenv("MYSQL_HOST")
DB_DATABASE = os.getenv("MYSQL_DATABASE")
engine = None # Inicializamos la variable

try:
    # --- PASO 1: CONECTARSE AL SERVIDOR Y CREAR LA BASE DE DATOS ---
    
    # 1.1 Conexión sin especificar la base de datos (Database=None)
    conn_no_db = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = conn_no_db.cursor()
    
    # 1.2 Crear la base de datos si no existe
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_DATABASE}")
    cursor.close()
    conn_no_db.close()
    
    print(f"✅ Base de datos '{DB_DATABASE}' creada o verificada en el servidor.")
    
    
    # --- PASO 2: CREAR EL ENGINE FINAL CON LA BASE DE DATOS ESPECIFICADA ---
    
    mysql_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_DATABASE}"
    engine = create_engine(mysql_url)
    
    print(f"✅ Conexión establecida a la base de datos MySQL: {DB_DATABASE}")

    # 1.3 Cargar el DataFrame de la Fase 1
    df_raw = pd.read_csv(CSV_FILENAME)
    df_raw = df_raw.rename(columns={'Año de lanzamiento': 'Año_lanzamiento'}) 
    
    print(f"✅ Datos cargados y columna renombrada: {len(df_raw)} filas.")
    print("\n--- ¡LISTO PARA EL MODELADO EN MYSQL! ---")

except ImportError:
    print("❌ ERROR: Asegúrate de que instalaste 'mysql-connector-python' o 'mysqlclient'.")
except Exception as e:
    print(f"❌ ERROR CRÍTICO DE CONEXIÓN: Verifica que tu servidor MySQL esté encendido y tus credenciales en '2.env' sean correctas. Detalle: {e}")
    engine = None

✅ Base de datos 'reggaeton_db' creada o verificada en el servidor.
✅ Conexión establecida a la base de datos MySQL: reggaeton_db
✅ Datos cargados y columna renombrada: 500 filas.

--- ¡LISTO PARA EL MODELADO EN MYSQL! ---


In [11]:
# --- CELDA 2 CORREGIDA: INSERCIÓN EN MYSQL (USANDO TIPOS DE SQLALCHEMY) ---

if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no se creó correctamente en la Celda 1.")
else:
    print("--- INICIANDO INSERCIÓN DE DATOS EN MYSQL ---")
    
    # 1. Preparar la tabla ARTISTAS
    df_artistas = df_raw[[
        'Artista', 
        'Playcount_LastFM', 
        'Listeners_LastFM', 
        'Biografia_Resumen'
    ]].drop_duplicates(subset=['Artista']).dropna(subset=['Artista'])
    
    df_artistas = df_artistas.replace({np.nan: None}) 

    # Insertar en MySQL. Usamos los objetos de tipo SQLAlchemy.
    df_artistas.to_sql('ARTISTAS', engine, if_exists='replace', index=False, dtype={
        # Usamos VARCHAR para el nombre del artista, especificando la longitud
        'Artista': VARCHAR(255),
        'Playcount_LastFM': BigInteger, # Usamos BigInteger para números grandes
        'Listeners_LastFM': BigInteger,
        'Biografia_Resumen': Text
    },
    # Añadimos una instrucción CREATE TABLE manual para la Clave Primaria (PK)
    # SQLAlchemy no maneja las PK complejas de esta forma en to_sql, así que se la inyectamos:
    schema=DB_DATABASE,
    chunksize=100)
    
    # Después de crear la tabla, ejecutamos el ALTER TABLE para añadir la PK
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE ARTISTAS ADD PRIMARY KEY (Artista)"))
        connection.commit()
    
    print(f"✅ Insertados {len(df_artistas)} artistas únicos en la tabla ARTISTAS.")

    # 2. Preparar la tabla ALBUMES
    df_albumes = df_raw[[
        'ID_Album', 
        'Nombre_Album', 
        'Año_Lanzamiento_Album', 
        'Artista_Principal_Album'
    ]].drop_duplicates(subset=['ID_Album']).dropna(subset=['ID_Album'])

    df_albumes = df_albumes.replace({np.nan: None}) 
    
    # Insertar en MySQL
    df_albumes.to_sql('ALBUMES', engine, if_exists='replace', index=False, dtype={
        'ID_Album': VARCHAR(50),
        'Nombre_Album': VARCHAR(255),
        'Año_Lanzamiento_Album': Integer,
        'Artista_Principal_Album': VARCHAR(255),
    }, chunksize=100)
    
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE ALBUMES ADD PRIMARY KEY (ID_Album)"))
        connection.commit()
    
    print(f"✅ Insertados {len(df_albumes)} álbumes únicos en la tabla ALBUMES.")


    # 3. Preparamos la tabla CANCIONES
    df_canciones = df_raw[[
        'ID_Spotify', 
        'Nombre', 
        'Año_lanzamiento', 
        'ID_Album'
    ]].drop_duplicates(subset=['ID_Spotify']).dropna(subset=['ID_Spotify'])

    df_canciones = df_canciones.replace({np.nan: None}) 

    # Insertar en MySQL
    df_canciones.to_sql('CANCIONES', engine, if_exists='replace', index=False, dtype={
        'ID_Spotify': VARCHAR(50),
        'Nombre': VARCHAR(255),
        'Año_lanzamiento': Integer,
        'ID_Album': VARCHAR(50)
    }, chunksize=100)
    
    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE CANCIONES ADD PRIMARY KEY (ID_Spotify)"))
        connection.commit()
        
    print(f"✅ Insertadas {len(df_canciones)} canciones únicas en la tabla CANCIONES.")

    # 4. Nota: Puedes añadir aquí las restricciones de CLAVE FORÁNEA (FOREIGN KEY) si lo deseas.
    # Por ahora, solo nos aseguramos de que los datos estén en la BBDD.
    
    print("\n--- FASE 2 (MODELADO) COMPLETADA. Datos insertados en MySQL. ---")

--- INICIANDO INSERCIÓN DE DATOS EN MYSQL ---


C:\Users\crist\AppData\Local\Temp\ipykernel_13664\3701334701.py:19: UserWarning: The provided table name 'ARTISTAS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_artistas.to_sql('ARTISTAS', engine, if_exists='replace', index=False, dtype={
C:\Users\crist\AppData\Local\Temp\ipykernel_13664\3701334701.py:49: UserWarning: The provided table name 'ALBUMES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_albumes.to_sql('ALBUMES', engine, if_exists='replace', index=False, dtype={


✅ Insertados 200 artistas únicos en la tabla ARTISTAS.
✅ Insertados 291 álbumes únicos en la tabla ALBUMES.
✅ Insertadas 500 canciones únicas en la tabla CANCIONES.

--- FASE 2 (MODELADO) COMPLETADA. Datos insertados en MySQL. ---


C:\Users\crist\AppData\Local\Temp\ipykernel_13664\3701334701.py:74: UserWarning: The provided table name 'CANCIONES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_canciones.to_sql('CANCIONES', engine, if_exists='replace', index=False, dtype={


In [ ]:
# --- PLANTILLA DE ANÁLISIS ---

# Asegúrate de que el 'engine' está disponible desde la Celda 1.
if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no está disponible.")
else:
    try:
        # 1. DEFINE AQUÍ TU NUEVA CONSULTA SQL
        query_mi_nuevo_analisis = """
        SELECT 
            T1.Columna_A,
            COUNT(T1.Columna_B) 
        FROM 
            TuTabla AS T1
        -- Usa INNER JOIN para conectar tus tablas (CANCIONES, ALBUMES, ARTISTAS)
        INNER JOIN ...
        WHERE 
            ...
        GROUP BY 
            ...
        """
        
        # 2. EJECUTA Y CONVIERTE EL RESULTADO A DATAFRAME
        df_nuevo_analisis = pd.read_sql_query(text(query_mi_nuevo_analisis), engine)
        
        print("--- RESULTADO DEL NUEVO ANÁLISIS ---")
        print(df_nuevo_analisis.to_string(index=False))
        
    except Exception as e:
        print(f"❌ ERROR al ejecutar la consulta SQL: {e}")

In [12]:
# --- FASE 3: ANÁLISIS 1 - POPULARIDAD PROMEDIO POR AÑO ---

import pandas as pd
from sqlalchemy.sql import text # Necesario para ejecutar la consulta

# Asegúrate de que el 'engine' está disponible desde la Celda 1.

if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no está disponible. Ejecuta la Celda 1 de la Fase 2.")
else:
    try:
        # Consulta SQL: Popularidad Promedio por Año
        # Unimos las tres tablas para obtener las métricas de LastFM (en ARTISTAS)
        # y agruparlas por el año de lanzamiento de la canción (en CANCIONES).
        
        query_popularidad_anual = """
        SELECT 
            T1.Año_lanzamiento AS Año,
            AVG(T3.Listeners_LastFM) AS Promedio_Listeners,
            AVG(T3.Playcount_LastFM) AS Promedio_Playcount,
            COUNT(T1.ID_Spotify) AS Total_Canciones_Muestra
        FROM 
            CANCIONES AS T1
        
        -- Conexión 1: De CANCIONES a ALBUMES (por ID_Album)
        INNER JOIN
            ALBUMES AS T2 ON T1.ID_Album = T2.ID_Album
        
        -- Conexión 2: De ALBUMES a ARTISTAS (por Artista_Principal_Album)
        INNER JOIN
            ARTISTAS AS T3 ON T2.Artista_Principal_Album = T3.Artista
        
        GROUP BY 
            T1.Año_lanzamiento
        ORDER BY 
            T1.Año_lanzamiento;
        """
        
        # Ejecutar la consulta y cargar resultados en un DataFrame
        df_popularidad_anual = pd.read_sql_query(text(query_popularidad_anual), engine)
        
        print("--- RESULTADO DEL ANÁLISIS DE POPULARIDAD POR AÑO (2000 vs 2024) ---")
        print(df_popularidad_anual.to_string(index=False)) # Imprime la tabla sin el índice
        
    except Exception as e:
        print(f"❌ ERROR al ejecutar la consulta SQL: {e}")

--- RESULTADO DEL ANÁLISIS DE POPULARIDAD POR AÑO (2000 vs 2024) ---
 Año  Promedio_Listeners  Promedio_Playcount  Total_Canciones_Muestra
2010         526381.6027        1.061992e+07                      224
2024         355720.3960        1.544301e+07                      250


In [15]:
# EJEMPLO: Artista con más canciones en la muestra

query_top_artista = """
SELECT 
    T2.Artista AS Nombre_Artista,
    COUNT(T1.ID_Spotify) AS Total_Canciones
FROM 
    CANCIONES AS T1
INNER JOIN 
    ALBUMES AS T3 ON T1.ID_Album = T3.ID_Album
INNER JOIN 
    ARTISTAS AS T2 ON T3.Artista_Principal_Album = T2.Artista
GROUP BY 
    Nombre_Artista
ORDER BY 
    Total_Canciones DESC
LIMIT 5;
"""

df_top_artista = pd.read_sql_query(text(query_top_artista), engine)
print("\n--- TOP 5 ARTISTAS POR NÚMERO DE CANCIONES ---")
print(df_top_artista.to_string(index=False))


--- TOP 5 ARTISTAS POR NÚMERO DE CANCIONES ---
         Nombre_Artista  Total_Canciones
         Wisin & Yandel               25
Colectivo Oriente Drums               24
               Don Omar               19
                 Plan B               16
            Cosculluela               13
